In [1]:
import pandas as pd
import os
import numpy as np

For reference on tracker setups: /home/csutter/DRIVE/weather_img_concatmodels/cnn_hrrr_fcsthr2/nested_trackerprep_data_experiments.ipynb

In [22]:
trackers_list = [
    "/home/csutter/DRIVE/dot/model_trackpaths/nestcv_5cat_twotrain_OT5_m0_T0V1.csv",
    "/home/csutter/DRIVE/dot/model_trackpaths/nestcv_5cat_twotrain_OT4_m3_T5V0.csv",
    "/home/csutter/DRIVE/dot/model_trackpaths/nestcv_5cat_twotrain_OT5_m4_T4V0.csv",
    "/home/csutter/DRIVE/dot/model_trackpaths/nestcv_5cat_twotrain_OT3_m4_T2V4.csv",
    "/home/csutter/DRIVE/dot/model_trackpaths/nestcv_5cat_twotrain_OT5_m3_T3V4.csv",
    "/home/csutter/DRIVE/dot/model_trackpaths/nestcv_5cat_twotrain_OT4_m0_T0V1.csv",
    "/home/csutter/DRIVE/dot/model_trackpaths/nestcv_5cat_twotrain_OT2_m0_T0V1.csv",
    "/home/csutter/DRIVE/dot/model_trackpaths/nestcv_5cat_twotrain_OT5_m2_T2V3.csv",
    "/home/csutter/DRIVE/dot/model_trackpaths/nestcv_5cat_twotrain_OT0_m3_T4V5.csv",
    "/home/csutter/DRIVE/dot/model_trackpaths/nestcv_5cat_twotrain_OT0_m4_T5V1.csv",
    "/home/csutter/DRIVE/dot/model_trackpaths/nestcv_5cat_twotrain_OT1_m3_T5V0.csv",
    "/home/csutter/DRIVE/dot/model_trackpaths/nestcv_5cat_twotrain_OT0_m2_T3V4.csv",
    "/home/csutter/DRIVE/dot/model_trackpaths/nestcv_5cat_twotrain_OT4_m2_T2V3.csv",
    "/home/csutter/DRIVE/dot/model_trackpaths/nestcv_5cat_twotrain_OT3_m1_T1V2.csv",
    "/home/csutter/DRIVE/dot/model_trackpaths/nestcv_5cat_twotrain_OT1_m2_T4V5.csv",
    "/home/csutter/DRIVE/dot/model_trackpaths/nestcv_5cat_twotrain_OT2_m1_T3V4.csv",
    "/home/csutter/DRIVE/dot/model_trackpaths/nestcv_5cat_twotrain_OT0_m1_T2V3.csv",
    "/home/csutter/DRIVE/dot/model_trackpaths/nestcv_5cat_twotrain_OT4_m4_T3V5.csv",
    "/home/csutter/DRIVE/dot/model_trackpaths/nestcv_5cat_twotrain_OT2_m4_T1V3.csv",
    "/home/csutter/DRIVE/dot/model_trackpaths/nestcv_5cat_twotrain_OT2_m2_T4V5.csv",
    "/home/csutter/DRIVE/dot/model_trackpaths/nestcv_5cat_twotrain_OT1_m4_T0V2.csv",
    "/home/csutter/DRIVE/dot/model_trackpaths/nestcv_5cat_twotrain_OT3_m3_T5V0.csv",
    "/home/csutter/DRIVE/dot/model_trackpaths/nestcv_5cat_twotrain_OT3_m0_T0V1.csv",
    "/home/csutter/DRIVE/dot/model_trackpaths/nestcv_5cat_twotrain_OT5_m1_T1V2.csv",
    "/home/csutter/DRIVE/dot/model_trackpaths/nestcv_5cat_twotrain_OT1_m1_T3V4.csv",
    "/home/csutter/DRIVE/dot/model_trackpaths/nestcv_5cat_twotrain_OT2_m3_T5V0.csv",
    "/home/csutter/DRIVE/dot/model_trackpaths/nestcv_5cat_twotrain_OT4_m1_T1V2.csv",
    "/home/csutter/DRIVE/dot/model_trackpaths/nestcv_5cat_twotrain_OT1_m0_T2V3.csv",
    "/home/csutter/DRIVE/dot/model_trackpaths/nestcv_5cat_twotrain_OT3_m2_T4V5.csv",
    "/home/csutter/DRIVE/dot/model_trackpaths/nestcv_5cat_twotrain_OT0_m0_T1V2.csv",
]

Shuffle

In [33]:
# Grab one tracker with foldnums in it (doesn't matter with OT and model it is, bc same foldnums are assigned to every 30 tracker)
sitespec = pd.read_csv("/home/csutter/DRIVE/dot/model_trackpaths/nestcv_5cat_twotrain_OT5_m0_T0V1.csv")
print(sitespec.columns)

# Randomly assign values 0 through 5
sitespec['shufflefold'] = np.random.randint(0, 6, size=len(sitespec))

for t in trackers_list:
    d = pd.read_csv(t)
    filename = os.path.basename(t)

    ### grab the assignments to map the foldnum to phases
    # assigments of phase WILL matter and have to be done by each of the 30, but want to make sure there is only ONE SINGLE fold assignment, so that must be done first before assigning phases by 30.
    # must grab these assignments before applying the changed foldnum!
    assignments = d[["foldnum", "innerPhase", "outerPhase"]].drop_duplicates().reset_index(drop=True)

    ### drop foldnum, innerPhase, and outerPhase columns, bc those will be redefined
    d = d.drop(columns=["foldnum", "innerPhase", "outerPhase"])

    ### apply the new shufflefold number from the assignments df and set it as foldnum
    d1 = d.merge(sitespec[['img_name','shufflefold']], how = "inner", on = "img_name")
    d2 = d1.rename(columns = {'shufflefold':'foldnum'})

    ### apply the assignments mapping foldnum to phases by merging
    d3 = d2.merge(assignments, how = "inner", on = "foldnum")

    # to print to see confurmation that foldnum is the same for a single example but phases differ based on which of the 30 trackers we're on
    # print(d3[d3["img_name"] =='I_81_at_Exit_50__Northbound__Skyline_6163_2022-02-06-19:00:17.jpg'][["foldnum","innerPhase"]])

    ### Save experiment trackers
    d3.to_csv(f"/home/csutter/DRIVE-clean/trackers_side_experiments/shuffle/{filename}")


# randomly assign foldnum to every row

Index(['Unnamed: 0.2', 'Unnamed: 0.1', 'innerPhase', 'Unnamed: 0',
       'outerPhase', 'img_name', 'img_orig', 'site', 'img_index', 'img_cat',
       'foldnum', 'ID', 'Latitude', 'Longitude', 'date', 'yr', 'mo', 'day',
       'time', 'yr_mo', 'timeofday', 'timeofevent', 'innerPhaseOrig'],
      dtype='object')


In [34]:
print(len(d3))

21489


Halved

In [20]:
# grab one have csv for reference on how to tie
example = pd.read_csv("/home/csutter/DRIVE/dot/model_trackpaths/nestcv_5cat_halved_OT0_m0_T1V2.csv")

example.head(3)

# read in the normal trackers and just inner join to keep only those that are included in the halved
for t in trackers_list:
    d = pd.read_csv(t)
    filename = os.path.basename(t)
    print(filename)
    dhalf = d.merge(example[["img_name"]], how = "inner", on = "img_name")
    dhalf.to_csv(f"/home/csutter/DRIVE-clean/trackers_side_experiments/halved/{filename}")

nestcv_5cat_twotrain_OT5_m0_T0V1.csv
nestcv_5cat_twotrain_OT4_m3_T5V0.csv
nestcv_5cat_twotrain_OT5_m4_T4V0.csv
nestcv_5cat_twotrain_OT3_m4_T2V4.csv
nestcv_5cat_twotrain_OT5_m3_T3V4.csv
nestcv_5cat_twotrain_OT4_m0_T0V1.csv
nestcv_5cat_twotrain_OT2_m0_T0V1.csv
nestcv_5cat_twotrain_OT5_m2_T2V3.csv
nestcv_5cat_twotrain_OT0_m3_T4V5.csv
nestcv_5cat_twotrain_OT0_m4_T5V1.csv
nestcv_5cat_twotrain_OT1_m3_T5V0.csv
nestcv_5cat_twotrain_OT0_m2_T3V4.csv
nestcv_5cat_twotrain_OT4_m2_T2V3.csv
nestcv_5cat_twotrain_OT3_m1_T1V2.csv
nestcv_5cat_twotrain_OT1_m2_T4V5.csv
nestcv_5cat_twotrain_OT2_m1_T3V4.csv
nestcv_5cat_twotrain_OT0_m1_T2V3.csv
nestcv_5cat_twotrain_OT4_m4_T3V5.csv
nestcv_5cat_twotrain_OT2_m4_T1V3.csv
nestcv_5cat_twotrain_OT2_m2_T4V5.csv
nestcv_5cat_twotrain_OT1_m4_T0V2.csv
nestcv_5cat_twotrain_OT3_m3_T5V0.csv
nestcv_5cat_twotrain_OT3_m0_T0V1.csv
nestcv_5cat_twotrain_OT5_m1_T1V2.csv
nestcv_5cat_twotrain_OT1_m1_T3V4.csv
nestcv_5cat_twotrain_OT2_m3_T5V0.csv
nestcv_5cat_twotrain_OT4_m1_T1V2.csv
n

Halved & shuffled

In [2]:
from glob import glob

In [37]:
# grab one have csv for reference on how to tie
example = pd.read_csv("/home/csutter/DRIVE/dot/model_trackpaths/nestcv_5cat_halved_OT0_m0_T1V2.csv")

example.head(3)

# here we have to specifically set the shuffle trackers list (which was atcually made above)
trackers_shuffle = glob("/home/csutter/DRIVE-clean/trackers_side_experiments/shuffle/*")
# read in the normal trackers and just inner join to keep only those that are included in the halved
for t in trackers_shuffle:
    d = pd.read_csv(t)
    filename = os.path.basename(t)
    print(filename)
    dhalf = d.merge(example[["img_name"]], how = "inner", on = "img_name")
    dhalf.to_csv(f"/home/csutter/DRIVE-clean/trackers_side_experiments/shuffle_halved/{filename}")

nestcv_5cat_twotrain_OT5_m0_T0V1.csv
nestcv_5cat_twotrain_OT4_m3_T5V0.csv
nestcv_5cat_twotrain_OT5_m4_T4V0.csv
nestcv_5cat_twotrain_OT3_m4_T2V4.csv
nestcv_5cat_twotrain_OT5_m3_T3V4.csv
nestcv_5cat_twotrain_OT4_m0_T0V1.csv
nestcv_5cat_twotrain_OT2_m0_T0V1.csv
nestcv_5cat_twotrain_OT5_m2_T2V3.csv
nestcv_5cat_twotrain_OT0_m3_T4V5.csv
nestcv_5cat_twotrain_OT0_m4_T5V1.csv
nestcv_5cat_twotrain_OT1_m3_T5V0.csv
nestcv_5cat_twotrain_OT0_m2_T3V4.csv
nestcv_5cat_twotrain_OT4_m2_T2V3.csv
nestcv_5cat_twotrain_OT3_m1_T1V2.csv
nestcv_5cat_twotrain_OT1_m2_T4V5.csv
nestcv_5cat_twotrain_OT2_m1_T3V4.csv
nestcv_5cat_twotrain_OT0_m1_T2V3.csv
nestcv_5cat_twotrain_OT4_m4_T3V5.csv
nestcv_5cat_twotrain_OT2_m4_T1V3.csv
nestcv_5cat_twotrain_OT2_m2_T4V5.csv
nestcv_5cat_twotrain_OT1_m4_T0V2.csv
nestcv_5cat_twotrain_OT3_m3_T5V0.csv
nestcv_5cat_twotrain_OT3_m0_T0V1.csv
nestcv_5cat_twotrain_OT5_m1_T1V2.csv
nestcv_5cat_twotrain_OT1_m1_T3V4.csv
nestcv_5cat_twotrain_OT2_m3_T5V0.csv
nestcv_5cat_twotrain_OT4_m1_T1V2.csv
n

Onetrain
- Rather than two training datasets, which makes the training smaller for each, have one massive training dataset, but we lose the "downstream" aspect. 


In [38]:
# these datasets were already set up.
# Note that these are entirely new phase assignments for a given foldnum (since now, with 6 folds, 4 are for training, 1 for val, and 1 for testing). Entirely different assignments
# Just copy over the csvs already set up in this way. 

# e.g. /home/csutter/DRIVE/dot/model_trackpaths/nestcv_5cat_onetrain_OT0_m0_T1V2.csv

Index(['Unnamed: 0.2', 'Unnamed: 0.1', 'innerPhase', 'Unnamed: 0',
       'outerPhase', 'img_name', 'img_orig', 'site', 'img_index', 'img_cat',
       'foldnum', 'ID', 'Latitude', 'Longitude', 'date', 'yr', 'mo', 'day',
       'time', 'yr_mo', 'timeofday', 'timeofevent', 'innerPhaseOrig'],
      dtype='object')


In [5]:
glob("/home/csutter/DRIVE-clean/trackers_side_experiments/shuffle_halved/*")

['/home/csutter/DRIVE-clean/trackers_side_experiments/shuffle_halved/nestcv_5cat_twotrain_OT5_m0_T0V1.csv',
 '/home/csutter/DRIVE-clean/trackers_side_experiments/shuffle_halved/nestcv_5cat_twotrain_OT4_m3_T5V0.csv',
 '/home/csutter/DRIVE-clean/trackers_side_experiments/shuffle_halved/nestcv_5cat_twotrain_OT5_m4_T4V0.csv',
 '/home/csutter/DRIVE-clean/trackers_side_experiments/shuffle_halved/nestcv_5cat_twotrain_OT3_m4_T2V4.csv',
 '/home/csutter/DRIVE-clean/trackers_side_experiments/shuffle_halved/nestcv_5cat_twotrain_OT5_m3_T3V4.csv',
 '/home/csutter/DRIVE-clean/trackers_side_experiments/shuffle_halved/nestcv_5cat_twotrain_OT4_m0_T0V1.csv',
 '/home/csutter/DRIVE-clean/trackers_side_experiments/shuffle_halved/nestcv_5cat_twotrain_OT2_m0_T0V1.csv',
 '/home/csutter/DRIVE-clean/trackers_side_experiments/shuffle_halved/nestcv_5cat_twotrain_OT5_m2_T2V3.csv',
 '/home/csutter/DRIVE-clean/trackers_side_experiments/shuffle_halved/nestcv_5cat_twotrain_OT0_m3_T4V5.csv',
 '/home/csutter/DRIVE-clean/